### Basic RAG
- .md files from data source
- Langchain-based RAG flow
- RecursiveChunking
- 

In [ ]:
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from chromadb import PersistentClient
from tqdm import tqdm
import numpy as np
from sklearn.manifold import TSNE
import glob
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import chromadb
from langchain_chroma import Chroma


load_dotenv(override=True)

In [2]:
import gradio as gr
print(gr.__version__)
# Gradio 6 onwards will generate hisotry in the format of [{"role": "user", "content": "..."}, ...]
# Gradio 5 and below will generate history in the format of [{"text": "..."}, ...]


5.49.1


In [3]:
DB_NAME = "chroma_db"
DATA_DIR = '../data' 
MODEL = "gpt-4o-mini"
TOP_K = 5

In [5]:
# load docs
# DATA_DIR = '../data'  # Remove the * - it's not part of the path

# docs=[]
# # Convert generator to list so we can use len() and iterate multiple times
# files = list(Path(DATA_DIR).glob('*'))
# print(f"Found {len(files)} files in the knowledge base")
# print(f"Files: {[file.name for file in files]}")

# for file in files:
#     if file.is_file():
#         print("Loading file: ", file)
#         with open(file, "r", encoding="utf-8") as f:
#             docs.append({"text": f.read(), "source": file.name, "type": file.suffix})

# print(f"\nLoaded {len(docs)} documents")
# print(docs)


documents = []

doc_type = os.path.basename(DATA_DIR) # get the name of the folder. employees, contracts, products, company
loader = DirectoryLoader(DATA_DIR, glob="*.md", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
folder_docs = loader.load()
for doc in folder_docs:
    doc.metadata["doc_type"] = doc_type
    doc.metadata["source"] = doc.metadata.get("source").split("/")[-1]
    documents.append(doc) # Has to be Document object !!!

print(documents)


[Document(metadata={'source': 'cv.md', 'doc_type': 'data'}, page_content='# LI BEIJI\n\n📞 +65 8432 9134  \n🔗 [LinkedIn]  \n📧 libeiji08121999@gmail.com  \n💻 [GitHub]  \n🌐 [Portfolio]\n\n---\n\n## SUMMARY\n\nPassionate Software Engineer and AI Innovator with a strong foundation in software engineering and applied AI. Experienced in designing and leading full-stack platforms and intelligent automation systems that drive real-world impact and unlock actionable AI insights.\n\n---\n\n## WORK EXPERIENCE\n\n### Software Engineer  \n**United Overseas Bank (UOB)**  \n*Aug 2023 – Present*\n\n- Led adoption and production rollout of **n8n**, enabling 10+ teams to build automation workflows and agentic solutions, saving **SGD 10k/month**.  \n- Designed and implemented **Prompt Template Hub** (React + Spring Boot) with Bitbucket integration, delivering version control and access control for **200+ prompt templates**.  \n- Built an in-house **Developer Portal** (Angular + Spring Boot) and contribute

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

print(f"Divided into {len(chunks)} chunks")
print(f"First chunk:\n\n{chunks[0]}")
print(f"Second chunk:\n\n{chunks[1]}")
print(f"Last chunk:\n\n{chunks[-1]}")

Divided into 30 chunks
First chunk:

page_content='# LI BEIJI

📞 +65 8432 9134  
🔗 [LinkedIn]  
📧 libeiji08121999@gmail.com  
💻 [GitHub]  
🌐 [Portfolio]

---

## SUMMARY' metadata={'source': 'cv.md', 'doc_type': 'data'}
Second chunk:

page_content='---

## SUMMARY

Passionate Software Engineer and AI Innovator with a strong foundation in software engineering and applied AI. Experienced in designing and leading full-stack platforms and intelligent automation systems that drive real-world impact and unlock actionable AI insights.

---' metadata={'source': 'cv.md', 'doc_type': 'data'}
Last chunk:

page_content='- After an injury-filled 2025 affecting his ankle and knee, he is fully recovered and planning to train for **Hyrox** in 2026. Despite not loving running, the full-body challenge excites him more than the discomfort, and he plans to begin structured training after Chinese New Year at BFT.' metadata={'source': 'hobby.md', 'doc_type': 'data'}


In [7]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

if os.path.exists(DB_NAME):
    Chroma(persist_directory=DB_NAME, embedding_function=embeddings,collection_name="my_collection").delete_collection()

vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=DB_NAME,
    collection_name="my_collection"
)

print(f"Vectorstore created with {vector_store._collection.count()} documents")

collection = vector_store._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

Vectorstore created with 30 documents
There are 30 vectors with 3,072 dimensions in the vector store


In [8]:
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings, collection_name="my_collection")
retriever = vectorstore.as_retriever()
retriever.invoke("What is Beiji's current job based on CV?")

[Document(id='77fa1c7c-f3a2-40c7-98d5-510f45f6f5b8', metadata={'source': 'hobby.md', 'doc_type': 'data'}, page_content='Beiji has been playing basketball since middle school, and it remains his favorite sport. During his time at Southeast University, he served as **captain of the Transportation Department basketball team**, leading training sessions, match preparation, and team strategy. Balancing competition with'),
 Document(id='4a22976a-14a6-4d20-bb9b-6a93ffe4842e', metadata={'doc_type': 'data', 'source': 'cv.md'}, page_content='# LI BEIJI\n\n📞 +65 8432 9134  \n🔗 [LinkedIn]  \n📧 libeiji08121999@gmail.com  \n💻 [GitHub]  \n🌐 [Portfolio]\n\n---\n\n## SUMMARY'),
 Document(id='91aed67f-d81f-41e8-8ba8-2f828c7cc8a3', metadata={'source': 'hobby.md', 'doc_type': 'data'}, page_content='Beiji began weightlifting in 2020 and has trained consistently since. By 2024, he achieved personal records of **175kg squat**, **190kg deadlift**, and **120kg bench press**. He appreciates weightlifting for it

In [10]:
# Check all documents in the collection
collection = vectorstore._collection
all_docs = collection.get(include=["documents", "metadatas"])

# Count by source
from collections import Counter
sources = [meta.get("source", "unknown") for meta in all_docs["metadatas"]]
print("Documents by source:", Counter(sources))

# Check for duplicates
doc_contents = all_docs["documents"]
print(f"Total documents: {len(doc_contents)}")
print(f"Unique documents: {len(set(doc_contents))}")

Documents by source: Counter({'cv.md': 17, 'hobby.md': 13})
Total documents: 30
Unique documents: 30


### RAG - Retrieval

In [11]:
from langchain_openai import ChatOpenAI


vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings, collection_name="my_collection")
retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})

llm=ChatOpenAI(model=MODEL) # Langchain wrapper for OpenAI

System_prompt = """
You are a helpful assistant that can answer questions about the user's CV and hobbies.
You are given a question and a context.
You need to answer the question based on the context.
Context:
{context}
"""

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, convert_to_messages


def fetch_context(query):
    """
    Fetch the context for the query from the vector store with Top K results
    """
    context = retriever.invoke(query)
    return context

def combine_all_user_questions(question: str, history: list) -> str:
    """
    Combine all user questions into a single question 
    to solve the issue that user ask "what did she do before"
    """
    prior = "\n".join(msg["content"] for msg in history if msg.get("role") == "user")
    
    # Handle empty prior case
    if prior:
        return prior + "\n" + question
    else:
        return question

def generate_answer(query, history: list):
    """
    Generate the answer for the query

    history from gradio is a list of dictionaries with keys "role" and "content" this is OpenAI format, need to convert it to the format that langchain can use
    """
    
    print(f"DEBUG - query type: {type(query)}, value: {query}")
    print(f"DEBUG - history type: {type(history)}, length: {len(history) if isinstance(history, list) else 'not a list'}")
    # ... rest of function

    # query = query["text"]
    
    # combine all user questions into a single question
    combined_question = combine_all_user_questions(query, history)
    context_docs = fetch_context(combined_question)
    context = "\n".join([doc.page_content for doc in context_docs])

    # convert history to langchain format
    messages = [SystemMessage(content=System_prompt.format(context=context))]
    print("history:", history)
    print("convert_to_messages(history):", convert_to_messages(history))
    print("messages before extend:", messages)
    messages.extend(convert_to_messages(history)) # system message + previous history
    print("messages after extend:", messages)
    messages.append(HumanMessage(content=query))
    print("messages after append human message:", messages)

    response = llm.invoke(messages)

    return response.content, context_docs

### Frontend with Gradio first for testing

In [ ]:
import gradio as gr
print(gr.__version__)

def format_context(context):
    result = "<h2 style='color: #ff7800;'>Relevant Context</h2>\n\n"
    for doc in context:
        result += f"<span style='color: #ff7800;'>Source: {doc.metadata['source']}</span>\n\n"
        result += doc.page_content + "\n\n"
    return result


def chat(history):
    """History is in messages format: [{"role": "user", "content": "..."}, ...]"""
    if not history:
        return history, ""
    
    last_message = history[-1]["content"]
    prior = history[:-1]
    answer, context = generate_answer(last_message, prior)
    
    # Create new history instead of modifying in place
    new_history = history + [{"role": "assistant", "content": answer}]
    return new_history, format_context(context)


def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]

theme = gr.themes.Soft(font=["Inter", "system-ui", "sans-serif"])

with gr.Blocks(title="Best Friend of Beiji", theme=theme) as ui:
    gr.Markdown("# 🏢 Best Friend of Beiji\nAsk me anything about Beiji!")

    with gr.Row():
        with gr.Column(scale=1):
            chatbot = gr.Chatbot(
                    label="💬 Conversation", height=600, type="messages", show_copy_button=True
                )
            message = gr.Textbox(
                label="Your Question",
                placeholder="Ask anything about Beiji...",
                show_label=False,
            )

        with gr.Column(scale=1):
            context_markdown = gr.Markdown(
                label="📚 Retrieved Context",
                value="*Retrieved context will appear here*",
                container=True,
                height=600,
            )

    message.submit(
        put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]
    ).then(chat, inputs=chatbot, outputs=[chatbot, context_markdown])

ui.launch(inbrowser=True)

5.49.1
* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


DEBUG - query type: <class 'str'>, value: what is beiji job
DEBUG - history type: <class 'list'>, length: 0
history: []
convert_to_messages(history): []
messages before extend: [SystemMessage(content="\nYou are a helpful assistant that can answer questions about the user's CV and hobbies.\nYou are given a question and a context.\nYou need to answer the question based on the context.\nContext:\nBeiji has been playing basketball since middle school, and it remains his favorite sport. During his time at Southeast University, he served as **captain of the Transportation Department basketball team**, leading training sessions, match preparation, and team strategy. Balancing competition with\n# LI BEIJI\n\n📞 +65 8432 9134  \n🔗 [LinkedIn]  \n📧 libeiji08121999@gmail.com  \n💻 [GitHub]  \n🌐 [Portfolio]\n\n---\n\n## SUMMARY\nBeiji began weightlifting in 2020 and has trained consistently since. By 2024, he achieved personal records of **175kg squat**, **190kg deadlift**, and **120kg bench press**.